In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import efficientnet.tfkeras as efn 

In [2]:
print(tf.__version__)

2.5.0


In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Num GPUs Available:  2
2 Physical GPUs, 2 Logical GPUs


In [4]:
train_dir = '/home/mikylab/cycleGan/melanomaImages/train/'
train_csv = pd.read_csv('/home/mikylab/cycleGan/clinic_csv/train.csv')

test_dir = '/home/mikylab/cycleGan/2016_test_data/'
test_csv = pd.read_csv('/home/mikylab/cycleGan/clinic_csv/ISBI2016_GroundTruth.csv')
train_csv.shape

test_2017_dir = '/home/mikylab/cycleGan/ISIC-2017_TestData/'
test_2017_csv = pd.read_csv('/home/mikylab/cycleGan/clinic_csv/ISIC-2017_Test_GroundTruth.csv')

In [5]:
MSKCC_csv = pd.read_csv('/home/mikylab/cycleGan/clinic_csv/MSKCC_contribution.csv')

In [6]:
MSKCC_frame = pd.DataFrame({'image_dir': MSKCC_csv['name'], 'target': MSKCC_csv['meta.clinical.benign_malignant']})

In [7]:
MSKCC_frame.target[MSKCC_frame.target == 'benign'] = 0
MSKCC_frame.target[MSKCC_frame.target == 'malignant'] = 1
MSKCC_frame.target = MSKCC_frame.target.apply(int)

In [8]:
MSKCC_mal_train = MSKCC_frame[MSKCC_frame['target']==1]
MSKCC_ben_train = MSKCC_frame[MSKCC_frame['target']==0]#.sample(n=500, random_state = 316)

MSKCC_ben_train2 = MSKCC_frame[MSKCC_frame['target']==0].sample(n=500, random_state = 316)
MSKCC_frame3 = MSKCC_frame.copy(deep=True)
MSKCC_frame2 =  pd.concat([MSKCC_mal_train, MSKCC_ben_train2], ignore_index=True, sort =False)
# MSKCC_frame2['image_dir'] = train_dir + MSKCC_frame2['image_dir'] + '.jpg'




MSKCC_frame =  pd.concat([MSKCC_mal_train, MSKCC_ben_train], ignore_index=True, sort =False)



In [9]:
# MSKCC_frame['image_dir'] = train_dir + MSKCC_frame['image_dir'] + '.jpg'

# MSKCC_frame3['image_dir'] = train_dir + MSKCC_frame3['image_dir'] + '.jpg'

In [10]:
BCN_csv = pd.read_csv('/home/mikylab/cycleGan/clinic_csv/BCN_contribution.csv')
BCN_frame = pd.DataFrame({'image_dir': BCN_csv['name'], 'target': BCN_csv['meta.clinical.benign_malignant']})
BCN_frame.target[BCN_frame.target == 'benign'] = 0
BCN_frame.target[BCN_frame.target == 'malignant'] = 1
BCN_frame.target = BCN_frame.target.apply(int)



In [11]:
# BCN_frame['image_dir'] = train_dir + BCN_frame['image_dir'] + '.jpg'

In [12]:
BCN_frame3 = BCN_frame.copy(deep=True)

In [13]:
BCN_mal_train = BCN_frame[BCN_frame['target']==1]
BCN_ben_train = BCN_frame[BCN_frame['target']==0]#.sample(n=500, random_state = 316)

BCN_ben_train2 = BCN_frame[BCN_frame['target']==0].sample(n=500, random_state = 316)
BCN_frame2 =  pd.concat([BCN_mal_train, BCN_ben_train2], ignore_index=True, sort =False)

BCN_frame =  pd.concat([BCN_mal_train, BCN_ben_train], ignore_index=True, sort =False)

In [14]:
neg, pos = np.bincount(train_csv['target'])
total = neg + pos
print('Melanoma Classification:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Melanoma Classification:
    Total: 33126
    Positive: 584 (1.76% of total)



In [15]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [16]:
mal_train = train_csv[train_csv['target']==1]
ben_train = train_csv[train_csv['target']==0]#.sample(n=2000, random_state = 316)
total_train =  pd.concat([mal_train, ben_train], ignore_index=True, sort =False)



In [17]:
#combine and change image_name to directory values
total_train =  pd.concat([mal_train, ben_train], ignore_index=True, sort =False)
total_train['image_name'] = train_dir + total_train['image_name'] + '.jpg'

#remove unnecessary columns
total_train = total_train.drop(['patient_id', 'sex', 'age_approx', 'anatom_site_general_challenge', 'diagnosis', 'benign_malignant'], axis = 1)

In [18]:
total_test = pd.DataFrame({'image_dir': test_csv['ISIC_0000003'], 'target': test_csv['0.0']})

In [19]:
total_test['image_dir'] = test_dir + total_test['image_dir'] + '.jpg'

In [20]:
test_2017_frame = pd.DataFrame({'image_dir': test_2017_csv['image_id'], 'target': test_2017_csv['melanoma']})

In [21]:
test_2017_frame['image_dir'] = test_2017_dir + test_2017_frame['image_dir'] + '.jpg'

In [24]:
neg, pos = np.bincount(data_frame_csv['target'])
total = neg + pos
print('Melanoma Classification:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

In [23]:
## from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(total_train['image_name'], total_train['target'], test_size = 0.2, shuffle = True, random_state = 316)
# train_gen = pd.DataFrame({'image_dir': x_train, 'target': y_train})
# val_gen = pd.DataFrame({'image_dir': x_val, 'target': y_val})


In [55]:
from sklearn.model_selection import train_test_split


x_train, x_val, y_train, y_val = train_test_split(MSKCC_ben_train2['image_dir'], MSKCC_ben_train2['target'], test_size = 0.2, shuffle = True, random_state = 316)
train_Benign = pd.DataFrame({'image_dir': x_train, 'target': y_train})
test_Benign = pd.DataFrame({'image_dir': x_val, 'target': y_val})


In [69]:
from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

import skimage.io as io
import os
save_path = '/home/mikylab/datasets/DRIT/trainA/'

def load_images(path, image_list, size=(256,256)):
    data_list = list()
    # enumerate filenames in directory, assume all are images
    for filename in image_list:
        # load and resize the image
        pixels = load_img(path + filename + '.jpg', target_size=size)
        pixels = img_to_array(pixels)
        io.imsave(os.path.join(save_path,"%s"%filename+'.jpg'),pixels)
#     return asarray(data_list)


# dataset path
# path = '/home/mikylab/datasets/cycleGan/'
path = '/home/mikylab/cycleGan/melanomaImages/train/'

# load dataset train images 
test = load_images(path, MSKCC_mal_train['image_dir'].to_numpy())

Lossy conversion from float32 to uint8. Range [3.0, 251.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [13.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [4.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [14.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [14.0, 220.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [10.0, 195.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [18.0, 240.0]. Convert image to uint8 prior to saving to suppress this warn

Lossy conversion from float32 to uint8. Range [17.0, 252.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [9.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [6.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [3.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [6.0, 249.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [1.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.

Lossy conversion from float32 to uint8. Range [14.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [12.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [5.0, 244.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [5.0, 252.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [3.0, 242.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 246.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [15.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [6.0, 255.0]. Convert image to uint8 prior to saving to suppress this warnin

Lossy conversion from float32 to uint8. Range [6.0, 246.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [3.0, 233.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [4.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [3.0, 187.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [2.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 254.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 249.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [11.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.

In [49]:
MSKCC_mal_train.shape[0] #+ MSKCC_ben_train.shape[0]


216

In [ ]:
array = np.intersect1d(BCN_ben_train['image_dir'].to_numpy(), BCN_ben_train2['image_dir'].to_numpy())

array2 = np.intersect1d(BCN_ben_train.index.to_numpy(), BCN_ben_train2.index.to_numpy())

# array = np.intersect1d(MSKCC_ben_train['image_dir'].to_numpy(), MSKCC_ben_train2['image_dir'].to_numpy())

# array2 = np.intersect1d(MSKCC_ben_train.index.to_numpy(), MSKCC_ben_train2.index.to_numpy())

In [59]:
#array #

In [51]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(BCN_frame['image_dir'], BCN_frame['target'], test_size = 0.2, shuffle = True, random_state = 316)
train_gen = pd.DataFrame({'image_dir': x_train, 'target': y_train})
val_gen = pd.DataFrame({'image_dir': x_val, 'target': y_val})


In [28]:
from sklearn.model_selection import train_test_split
x_train2, x_val2, y_train2, y_val2 = train_test_split(BCN_frame2['image_dir'], BCN_frame2['target'], test_size = 0.2, shuffle = True, random_state = 316)
train_gen2 = pd.DataFrame({'image_dir': x_train2, 'target': y_train2})
val_gen2 = pd.DataFrame({'image_dir': x_val2, 'target': y_val2})



In [29]:
val_gen.shape#[val_gen.target == 1]['image_dir'].to_numpy().shape

(1463, 2)

In [30]:
val_gen2.shape

(136, 2)

In [36]:
array = np.intersect1d(BCN_ben_train['image_dir'].to_numpy(), BCN_ben_train2['image_dir'].to_numpy())

array2 = np.intersect1d(BCN_ben_train.index.to_numpy(), BCN_ben_train2.index.to_numpy())

# array = np.intersect1d(MSKCC_ben_train['image_dir'].to_numpy(), MSKCC_ben_train2['image_dir'].to_numpy())

# array2 = np.intersect1d(MSKCC_ben_train.index.to_numpy(), MSKCC_ben_train2.index.to_numpy())

In [34]:
#ISIC_0200921

In [47]:
BCN_frame3.iloc[43][0]

'ISIC_0200921'

In [40]:
test_bcn = BCN_frame3.drop(BCN_frame3.index[array2])


# test_mskcc = MSKCC_frame3.drop(MSKCC_frame3.index[array2])


In [62]:
benign_frame = test_bcn[test_bcn.target == 0]
x_train3, x_val3, y_train3, y_val3 = train_test_split(benign_frame['image_dir'], benign_frame['target'], test_size = 0.19, shuffle = True, random_state = 316)
train_gen3 = pd.DataFrame({'image_dir': x_train3, 'target': y_train3})
val_gen3 = pd.DataFrame({'image_dir': x_val3, 'target': y_val3})


# benign_frame = test_mskcc[test_mskcc.target == 0]
# x_train3, x_val3, y_train3, y_val3 = train_test_split(benign_frame['image_dir'], benign_frame['target'], test_size = 0.25, shuffle = True, random_state = 316)
# train_gen3 = pd.DataFrame({'image_dir': x_train3, 'target': y_train3})
# val_gen3 = pd.DataFrame({'image_dir': x_val3, 'target': y_val3})

In [63]:
val_gen4 = pd.concat([val_gen2, val_gen3], ignore_index=True, sort =False)
train_gen4 = pd.concat([train_gen2, train_gen3], ignore_index=True, sort =False)


In [43]:
val_gen4[val_gen4.target == 0].shape[0] + train_gen4[train_gen4.target == 0].shape[0]

np.intersect1d(val_gen4['image_dir'].to_numpy(), train_gen4['image_dir'].to_numpy()).shape

(0,)

In [71]:
val_gen4.duplicated().value_counts()

False    1397
dtype: int64

In [73]:
train_gen4.to_csv('train_BCN.csv', index = False)

In [78]:
data_frame_csv = pd.read_csv('train_BCN.csv')

In [61]:
data_frame_csv[data_frame_csv.target == 0].shape

(1972, 2)

In [59]:
data_frame_csv.shape

(9093, 2)

In [58]:
np.intersect1d(data_frame_csv['image_dir'].to_numpy(), train_gen4['image_dir'].to_numpy()).shape

(9093,)

In [79]:
neg, pos = np.bincount(data_frame_csv['target'])
total = neg + pos
print('Melanoma Classification:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Melanoma Classification:
    Total: 1330
    Positive: 34 (2.56% of total)



In [221]:
BCN_frame.shape#[BCN_frame.target == 1].shape

(7311, 2)

In [161]:
np.intersect1d(test_bcn['image_dir'].to_numpy(), BCN_ben_train2['image_dir'].to_numpy()).shape

(0,)

In [30]:
np.setdiff1d(BCN_ben_train['image_dir'].to_numpy(), val_gen2['image_dir'].to_numpy()).shape

(7030,)

In [77]:
np.intersect1d(train_gen2['image_dir'].to_numpy(), val_gen[val_gen.target == 1]['image_dir'].to_numpy()).shape




(33,)

In [24]:
# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(BCN_frame['image_dir'], BCN_frame['target'], test_size = 0.2, shuffle = True, random_state = 316)
# train_gen = pd.DataFrame({'image_dir': x_train, 'target': y_train})
# val_gen = pd.DataFrame({'image_dir': x_val, 'target': y_val})



In [95]:
neg, pos = np.bincount(val_gen['target'])
total = neg + pos
print('Melanoma Classification:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Melanoma Classification:
    Total: 2222
    Positive: 35 (1.58% of total)



In [78]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.72
Weight for class 1: 1.66
